# Surprise homework

In [52]:
import io 
import os
import pandas as pd
from collections import defaultdict
from surprise import SVD
from surprise import Dataset
from surprise import KNNWithMeans
from surprise import NormalPredictor
from surprise.model_selection import cross_validate
from surprise.model_selection import KFold

### Load data

In [53]:
data = Dataset.load_builtin('ml-100k')

### Using algorithms

In [54]:
cross_validate(NormalPredictor(), data, measures=['RMSE'], verbose=True)

Evaluating RMSE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.5160  1.5275  1.5215  1.5297  1.5102  1.5210  0.0072  
Fit time          0.09    0.11    0.11    0.11    0.11    0.11    0.01    
Test time         0.09    0.09    0.09    0.09    0.33    0.14    0.10    


{'test_rmse': array([1.51597776, 1.52751479, 1.52154419, 1.52971964, 1.51021738]),
 'fit_time': (0.08976459503173828,
  0.10771608352661133,
  0.10771441459655762,
  0.10771632194519043,
  0.11270022392272949),
 'test_time': (0.09272050857543945,
  0.09075331687927246,
  0.09374594688415527,
  0.09176111221313477,
  0.3321084976196289)}

In [55]:
cross_validate(KNNWithMeans(k = 30, sim_options={'name': 'cosine'}), data, measures=['RMSE'], verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9541  0.9571  0.9609  0.9605  0.9548  0.9575  0.0028  
Fit time          1.40    1.48    1.39    1.40    1.37    1.41    0.04    
Test time         2.33    2.41    2.43    2.26    2.54    2.39    0.10    


{'test_rmse': array([0.95411625, 0.95706798, 0.96089791, 0.96047841, 0.95484186]),
 'fit_time': (1.404280424118042,
  1.4770538806915283,
  1.3882887363433838,
  1.3992249965667725,
  1.367349624633789),
 'test_time': (2.326742649078369,
  2.410553216934204,
  2.4334914684295654,
  2.256991386413574,
  2.5381829738616943)}

In [56]:
cross_validate(KNNWithMeans(k = 30, sim_options={'name': 'pearson'}), data, measures=['RMSE'], verbose=True)

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9478  0.9521  0.9485  0.9566  0.9555  0.9521  0.0036  
Fit time          1.81    1.87    1.86    1.94    1.78    1.85    0.05    
Test time         2.27    2.20    2.38    2.30    2.20    2.27    0.07    


{'test_rmse': array([0.94782705, 0.95212977, 0.94848268, 0.95662979, 0.95550103]),
 'fit_time': (1.8071339130401611,
  1.8670198917388916,
  1.863027811050415,
  1.9378411769866943,
  1.779249668121338),
 'test_time': (2.2669646739959717,
  2.2031049728393555,
  2.3826212882995605,
  2.304823637008667,
  2.2031068801879883)}

In [57]:
cross_validate(SVD(), data, measures=['RMSE'], verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9345  0.9389  0.9353  0.9334  0.9366  0.9358  0.0019  
Fit time          3.82    3.76    3.90    3.82    3.88    3.84    0.05    
Test time         0.09    0.12    0.10    0.10    0.11    0.11    0.01    


{'test_rmse': array([0.93450899, 0.93888864, 0.93532048, 0.93344251, 0.93662504]),
 'fit_time': (3.8227832317352295,
  3.757993459701538,
  3.9035956859588623,
  3.823807716369629,
  3.8766369819641113),
 'test_time': (0.09474396705627441,
  0.1166837215423584,
  0.0997314453125,
  0.10270237922668457,
  0.11469268798828125)}

### Calculate precision@k and recall@k

In [75]:
def precision_recall_at_k(predictions, k=10, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls


data = Dataset.load_builtin('ml-100k')
kf = KFold(n_splits=5)
algo = SVD()

for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=3.52)

    # Precision and recall can then be averaged over all users
    print(sum(prec for prec in precisions.values()) / len(precisions))
    print(sum(rec for rec in recalls.values()) / len(recalls))

0.723662770102727
0.4056579507726084
0.724142806645458
0.40012421163107365
0.7233545647558397
0.40073055985172423
0.7232767762460244
0.40827686661455276
0.7249202975557921
0.3986737566889412


### Predict

In [76]:
def get_top_n(predictions, n=10):
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

# обратите внимание на функцию build_anti_testset
trainset = data.build_full_trainset()

algo = SVD()
algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=5)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

196 ['64', '178', '603', '408', '190']
186 ['480', '603', '479', '174', '199']
22 ['98', '183', '64', '318', '480']
244 ['483', '286', '175', '178', '124']
166 ['318', '114', '1', '132', '50']
298 ['661', '64', '251', '408', '169']
115 ['408', '285', '135', '923', '134']
253 ['313', '169', '174', '657', '493']
305 ['519', '515', '211', '177', '603']
6 ['603', '657', '923', '114', '641']
62 ['192', '919', '169', '408', '187']
286 ['64', '187', '318', '1142', '12']
200 ['144', '659', '64', '657', '186']
210 ['169', '408', '318', '285', '12']
224 ['174', '121', '50', '210', '496']
303 ['197', '169', '180', '527', '272']
122 ['515', '169', '657', '483', '603']
194 ['603', '170', '42', '480', '272']
291 ['483', '127', '318', '357', '169']
234 ['59', '408', '169', '302', '57']
119 ['479', '114', '178', '603', '1007']
167 ['427', '64', '12', '272', '191']
299 ['357', '223', '124', '9', '1449']
308 ['302', '114', '251', '315', '189']
95 ['318', '272', '222', '603', '166']
38 ['143', '121', '25

In [77]:
file_path = os.path.expanduser('~/.surprise_data/ml-100k/ml-100k/u.item')
films = pd.read_csv(file_path, sep="|", encoding='ansi', usecols=[0,1,2], names=['id','name', 'date'])
print("User 30")
for id, score in top_n['30']:
    name = films.at[int(id) - 1, 'name']
    date = films.at[int(id) - 1, 'date']
    print("{:<4} ({:<60} {:<11}) {:<10.3f}".format(id, name, date, score))

User 30
64   (Shawshank Redemption, The (1994)                             01-Jan-1994) 5.000     
318  (Schindler's List (1993)                                      01-Jan-1993) 4.961     
408  (Close Shave, A (1995)                                        28-Apr-1996) 4.839     
114  (Wallace & Gromit: The Best of Aardman Animation (1996)       05-Apr-1996) 4.801     
513  (Third Man, The (1949)                                        01-Jan-1949) 4.752     
